In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MaxNLocator

pd.set_option('float_format', '{:6.2f}'.format)

ModuleNotFoundError: No module named 'matplotlib'

In [18]:
Kenya_ex = pd.read_csv("Kenya2012_2022exportsHS2-6.csv")
Kenya_im = pd.read_csv("Kenya2012_2022importHS2-6.csv")
Kenya_GDP= pd.read_csv("GDP.csv")
Kenya_ex

,HS2,HS2 ID,HS4,HS4 ID,HS6,HS6 ID,Section,Section ID,Trade Value,Year
0,Live animals,101,Horses,10101,Pure-bred Breeding Horses,1010111,Animal Products,1,3846.0,2012
1,Live animals,101,Horses,10101,Non-breeding Horses,1010119,Animal Products,1,13300.0,2012
2,Live animals,101,Horses,10101,"Live Donkeys, Mules, and Crossbreeds",1010120,Animal Products,1,9355.0,2012
3,Live animals,101,Bovine,10102,Pure-bred Breeding Cattle,1010210,Animal Products,1,56766.0,2012
4,Live animals,101,Bovine,10102,Non-breeding Cattle,1010290,Animal Products,1,2624789.0,2012
...,...,...,...,...,...,...,...,...,...,...
40159,Art & antiques,2197,Prints,219702,"Original engravings, prints and lithographs",21970200,Arts and Antiques,21,1849.0,2022
40160,Art & antiques,2197,Sculptures,219703,"Original sculptures and statuary, in any material",21970300,Arts and Antiques,21,638981.0,2022
40161,Art & antiques,2197,Revenue Stamps,219704,"Used postage and revenue stamps, first day cov...",21970400,Arts and Antiques,21,2507.0,2022
40162,Art & antiques,2197,Collector's Items,219705,Collections and collectors pieces,21970500,Arts and Antiques,21,391474.0,2022


In [20]:
Kenya_im

,HS2,HS2 ID,HS4,HS4 ID,HS6,HS6 ID,Section,Section ID,Trade Value,Year
0,Live animals,101,Horses,10101,Pure-bred Breeding Horses,1010111,Animal Products,1,185562.0,2012
1,Live animals,101,Horses,10101,Non-breeding Horses,1010119,Animal Products,1,16239.0,2012
2,Live animals,101,Horses,10101,"Live Donkeys, Mules, and Crossbreeds",1010120,Animal Products,1,8204.0,2012
3,Live animals,101,Bovine,10102,Non-breeding Cattle,1010290,Animal Products,1,12187.0,2012
4,Live animals,101,Sheep and Goats,10104,Live Sheep,1010410,Animal Products,1,5.0,2012
...,...,...,...,...,...,...,...,...,...,...
46363,Art & antiques,2197,Prints,219702,"Original engravings, prints and lithographs",21970200,Arts and Antiques,21,12479.0,2022
46364,Art & antiques,2197,Sculptures,219703,"Original sculptures and statuary, in any material",21970300,Arts and Antiques,21,68007.0,2022
46365,Art & antiques,2197,Revenue Stamps,219704,"Used postage and revenue stamps, first day cov...",21970400,Arts and Antiques,21,4810.0,2022
46366,Art & antiques,2197,Collector's Items,219705,Collections and collectors pieces,21970500,Arts and Antiques,21,82799.0,2022


In [22]:
Kenya_GDP

,Country Name,Country Code,Series Name,Series Code,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],GDP of Kenya
0,Kenya,KEN,GDP (current US$),NY.GDP.MKTP.CD,56396704672,61671440408,68285796514,70120446897,74815144164,82036510877,92202979985,1.003780e+11,1.006580e+11,1.097040e+11,1.134200e+11,6.986871


In [24]:
Kenya_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40164 entries, 0 to 40163
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HS2          40164 non-null  object 
 1   HS2 ID       40164 non-null  int64  
 2   HS4          40164 non-null  object 
 3   HS4 ID       40164 non-null  int64  
 4   HS6          40164 non-null  object 
 5   HS6 ID       40164 non-null  int64  
 6   Section      40164 non-null  object 
 7   Section ID   40164 non-null  int64  
 8   Trade Value  40164 non-null  float64
 9   Year         40164 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 3.1+ MB


In [26]:
Kenya_ex.mean(numeric_only=True)

HS2 ID         1.133208e+03
HS4 ID         1.133348e+05
HS6 ID         1.133352e+07
Section ID     1.077791e+01
Trade Value    1.766068e+06
Year           2.016976e+03
dtype: float64

In [28]:
Kenya_ex.nunique(axis=0)

HS2               96
HS2 ID            96
HS4             1212
HS4 ID          1212
HS6             4514
HS6 ID          4522
Section           21
Section ID        21
Trade Value    31600
Year              11
dtype: int64

In [30]:
Kenya_ex.describe()

,HS2 ID,HS4 ID,HS6 ID,Section ID,Trade Value,Year
count,40164.000000,40164.000000,4.016400e+04,40164.000000,4.016400e+04,40164.000000
mean,1133.208147,113334.836495,1.133352e+07,10.777911,1.766068e+06,2016.975700
std,562.378719,56242.164773,5.624220e+06,5.349659,2.452216e+07,3.144862
min,101.000000,10101.000000,1.010111e+06,1.000000,1.000000e+00,2012.000000
25%,629.000000,62933.000000,6.293390e+06,6.000000,3.592750e+03,2014.000000
50%,1161.000000,116104.000000,1.161045e+07,11.000000,2.906650e+04,2017.000000
75%,1684.000000,168420.000000,1.684210e+07,16.000000,1.989582e+05,2020.000000
max,2197.000000,219706.000000,2.197060e+07,21.000000,1.390005e+09,2022.000000


In [32]:
Kenya_ex['Total_Industry_Count'] = Kenya_ex.groupby('Section ID')['Section'].transform('count')
Kenya_ex.rename(columns={'Trade Value': 'Trade_Value', 'Section ID': 'Section_ID', 'HS2 ID': 'HS2_ID', 'HS4 ID': 'HS4_ID','HS6 ID': 'HS6_ID' })

,HS2,HS2_ID,HS4,HS4_ID,HS6,HS6_ID,Section,Section_ID,Trade_Value,Year,Total_Industry_Count
0,Live animals,101,Horses,10101,Pure-bred Breeding Horses,1010111,Animal Products,1,3846.0,2012,1425
1,Live animals,101,Horses,10101,Non-breeding Horses,1010119,Animal Products,1,13300.0,2012,1425
2,Live animals,101,Horses,10101,"Live Donkeys, Mules, and Crossbreeds",1010120,Animal Products,1,9355.0,2012,1425
3,Live animals,101,Bovine,10102,Pure-bred Breeding Cattle,1010210,Animal Products,1,56766.0,2012,1425
4,Live animals,101,Bovine,10102,Non-breeding Cattle,1010290,Animal Products,1,2624789.0,2012,1425
...,...,...,...,...,...,...,...,...,...,...,...
40159,Art & antiques,2197,Prints,219702,"Original engravings, prints and lithographs",21970200,Arts and Antiques,21,1849.0,2022,77
40160,Art & antiques,2197,Sculptures,219703,"Original sculptures and statuary, in any material",21970300,Arts and Antiques,21,638981.0,2022,77
40161,Art & antiques,2197,Revenue Stamps,219704,"Used postage and revenue stamps, first day cov...",21970400,Arts and Antiques,21,2507.0,2022,77
40162,Art & antiques,2197,Collector's Items,219705,Collections and collectors pieces,21970500,Arts and Antiques,21,391474.0,2022,77


In [34]:
Kenya_section = Kenya_ex[Kenya_ex['Section'] == "Animal Products"]
Kenya_section

,HS2,HS2 ID,HS4,HS4 ID,HS6,HS6 ID,Section,Section ID,Trade Value,Year,Total_Industry_Count
0,Live animals,101,Horses,10101,Pure-bred Breeding Horses,1010111,Animal Products,1,3846.0,2012,1425
1,Live animals,101,Horses,10101,Non-breeding Horses,1010119,Animal Products,1,13300.0,2012,1425
2,Live animals,101,Horses,10101,"Live Donkeys, Mules, and Crossbreeds",1010120,Animal Products,1,9355.0,2012,1425
3,Live animals,101,Bovine,10102,Pure-bred Breeding Cattle,1010210,Animal Products,1,56766.0,2012,1425
4,Live animals,101,Bovine,10102,Non-breeding Cattle,1010290,Animal Products,1,2624789.0,2012,1425
...,...,...,...,...,...,...,...,...,...,...,...
36750,"Animal originated products, n.e.s.",105,Coral and Shells,10508,Coral and Shells,1050800,Animal Products,1,176158.0,2022,1425
36751,"Animal originated products, n.e.s.",105,Pharmaceutical Animal Products,10510,Pharmaceutical Animal Products,1051000,Animal Products,1,269290.0,2022,1425
36752,"Animal originated products, n.e.s.",105,Other Inedible Animal Products,10511,Bovine Semen,1051110,Animal Products,1,245477.0,2022,1425
36753,"Animal originated products, n.e.s.",105,Other Inedible Animal Products,10511,Inedible Fish and Crustaceans,1051191,Animal Products,1,60427.0,2022,1425


In [36]:
Kenya_ex_2012 = Kenya_ex[Kenya_ex.Year == 2012].copy()
Kenya_ex_2012

,HS2,HS2 ID,HS4,HS4 ID,HS6,HS6 ID,Section,Section ID,Trade Value,Year,Total_Industry_Count
0,Live animals,101,Horses,10101,Pure-bred Breeding Horses,1010111,Animal Products,1,3846.0,2012,1425
1,Live animals,101,Horses,10101,Non-breeding Horses,1010119,Animal Products,1,13300.0,2012,1425
2,Live animals,101,Horses,10101,"Live Donkeys, Mules, and Crossbreeds",1010120,Animal Products,1,9355.0,2012,1425
3,Live animals,101,Bovine,10102,Pure-bred Breeding Cattle,1010210,Animal Products,1,56766.0,2012,1425
4,Live animals,101,Bovine,10102,Non-breeding Cattle,1010290,Animal Products,1,2624789.0,2012,1425
...,...,...,...,...,...,...,...,...,...,...,...
3545,Art & antiques,2197,Prints,219702,"Original engravings, prints and lithographs",21970200,Arts and Antiques,21,208799.0,2012,77
3546,Art & antiques,2197,Sculptures,219703,"Original sculptures and statuary, in any material",21970300,Arts and Antiques,21,772108.0,2012,77
3547,Art & antiques,2197,Revenue Stamps,219704,"Used postage and revenue stamps, first day cov...",21970400,Arts and Antiques,21,2975.0,2012,77
3548,Art & antiques,2197,Collector's Items,219705,Collections and collectors pieces,21970500,Arts and Antiques,21,38048.0,2012,77


In [ ]:
Kenya_ex_2012.nlargest(10, 'Trade Value')

In [ ]:
Kenya_ex_2022 = Kenya_ex[Kenya_ex.Year == 2022].copy()
Kenya_ex_2022


In [ ]:
Kenya_ex_2022.nlargest(10,'Trade Value') 

In [ ]:
def most_traded(year, section, n):
    return Kenya_ex[(Kenya_ex.Year == year) &  (Kenya_ex.Section == section)].nlargest(n, 'Trade Value')

In [ ]:
most_traded(2018, 'Animal Products', 5)

In [ ]:
most_traded(2022,'Animal Products', 5)

In [ ]:
Kenya_section = Kenya_ex[Kenya_ex.Section == 'Animal Products'].copy()
Kenya_section

In [ ]:
# Group by year and section to find the total trade value per section per year
# This will show the Year and Section as the multi-level indices
yearly_exports = Kenya_ex.groupby(['Year', 'Section'])['Trade Value'].sum().reset_index()

# Pivot the data to get a better format for plotting
pivot_exports = yearly_exports.pivot(index='Year', columns='Section', values='Trade Value').fillna(0)

In [ ]:
pivot_exports

In [ ]:
# Create a correlation matrix using pandas

# Exclude non-numeric columns
corr_matrix = pivot_exports.select_dtypes(include=[ 'float64', 'int64'])

# Now compute the correlation matrix
corr_matrix= pivot_exports.corr()
corr_matrix

In [ ]:

f = plt.figure(figsize=(10, 10)) # figure size 
plt.matshow(pivot_exports.corr(), fignum=f.number) # type of plot
plt.xticks(range(pivot_exports.shape[1]), pivot_exports.columns, fontsize=15, rotation=90) # x axis labels
plt.yticks(range(pivot_exports.shape[1]), pivot_exports.columns, fontsize=15) # y axis labels
cb = plt.colorbar() # add a colour legend (called colorbar)
cb.ax.tick_params(labelsize=14) # add font size
plt.title('Correlation Matrix', fontsize=14) # add title
plt.show() 

In [ ]:
corr_matrix = pivot_exports.corr()
f, ax = plt.subplots(figsize=(16, 12))
# Create the heatmap without annotations
sns.heatmap(corr_matrix, annot=False, ax=ax, cmap='coolwarm', linewidths=0.5)

# Define a threshold to distinguish dark and light cells
threshold = corr_matrix.max().max() / 2  # This is an example threshold

# Manually annotate each cell with the correlation coefficient
for i in range(len(corr_matrix.columns)):
    for j in range(len(corr_matrix.columns)):
        value = corr_matrix.iloc[i, j]
        text_color = "black" if value < threshold else "white"
        ax.text(j + 0.5, i + 0.5, f"{value:.2f}",
                horizontalalignment='center',
                verticalalignment='center',
                color=text_color)



# Now we set the title
plt.title('Correlation Matrix', fontsize=14)

# and finally show the plot
plt.show()

In [ ]:
most_exported = pivot_exports.idxmax(axis=1)
least_exported = pivot_exports.idxmin(axis=1)

fig, ax = plt.subplots(2, 1, figsize=(14, 12))

# Most exported sections over the years
sns.lineplot(data=pivot_exports[most_exported.unique()], ax=ax[0])
ax[0].set_title('Most Exported Sections by Year')
ax[0].set_ylabel('Trade Value')
ax[0].set_xlabel('Year')
ax[0].legend(title='Section')

# Least exported sections over the years
sns.lineplot(data=pivot_exports[least_exported.unique()], ax=ax[1])
ax[1].set_title('Least Exported Sections by Year')
ax[1].set_ylabel('Trade Value')
ax[1].set_xlabel('Year')
ax[1].legend(title='Section')

plt.tight_layout()
plt.show()

most_exported, least_exported

In [ ]:
industry_counts = df.groupby('Section')['Section'].count()

# Define the data
data = {
    'Animal Hides': 33,
    'Animal Products': 55,
    'Animal and Vegetable Bi-Products': 11,
    'Arts and Antiques': 11,
    'Chemical Products': 121,
    'Foodstuffs': 99,
    'Footwear and Headwear': 44,
    'Instruments': 33,
    'Machines': 22,
    'Metals': 121,
    'Mineral Products': 33,
    'Miscellaneous': 33,
    'Paper Goods': 33,
    'Plastics and Rubbers': 22,
    'Precious Metals': 11,
    'Stone And Glass': 33,
    'Textiles': 154,
    'Transportation': 44,
    'Vegetable Products': 99,
    'Weapons': 11,
    'Wood Products': 33
}

# Create lists for section names and values
sections = list(data.keys())
values = list(data.values())

# Plotting
plt.figure(figsize=(10, 6))
plt.barh(sections, values, color='skyblue')
plt.xlabel('Number of Items')
plt.ylabel('Section')
plt.title('Number of Items per Section')
plt.gca().invert_yaxis()  # Invert y-axis to display sections from top to bottom
plt.tight_layout()

# Show the plot
plt.show()
industry_counts

In [ ]:
total_products = industry_counts.sum()
print("Total Products across all Industries:", total_products)

#This is the Industry Trade value

In [ ]:
industry_trade_value = df.groupby('Section')['Trade Value'].sum()
highest_trade_value_industry = industry_trade_value.idxmax()
highest_trade_value = industry_trade_value.max()
lowest_trade_value_industry = industry_trade_value.idxmin()
lowest_trade_value = industry_trade_value.min()

#Highest and lowest value of trade value within an Industry


print(f"The industry with the lowest trade value is '{lowest_trade_value_industry}' with a total trade value of {lowest_trade_value}.")
print(f"The industry with the highest trade value is '{highest_trade_value_industry}' with a total trade value of {highest_trade_value}.")

In [ ]:
df.describe(include="all")

In [ ]:
raw = pd.read_csv("Kenya_goods_export_2012-2022_HS2.csv")
# Strip whitespace from column names
raw.columns = [x.strip() for x in raw.columns]

# Print shape and data types
print(raw.shape)
print(raw.dtypes)

In [ ]:
print(raw.describe().T[['min', 'mean', 'max', 'std']])

In [ ]:
df = Kenya 

# Assuming df contains the trade data for Kenya

# Group the data by 'Year' and calculate the sum, mean, minimum, and maximum of the trade values for each year
yearly_trade_sum = df.groupby('Year')['Trade Value'].sum()
mean_trade_value_kenya = df.groupby('Year')['Trade Value'].mean()
min_trade_value_kenya = df.groupby('Year')['Trade Value'].min()
max_trade_value_kenya = df.groupby('Year')['Trade Value'].max()

# Create a DataFrame to hold the trade summary information
trade_summary = pd.DataFrame({
    'Yearly Trade Sum': yearly_trade_sum,
    'Mean Trade Value': mean_trade_value_kenya,
    'Min Trade Value': min_trade_value_kenya,
    'Max Trade Value': max_trade_value_kenya
})

# Print the trade summary DataFrame
(trade_summary)

# Print the mean trade value for Kenya over the entire period



In [ ]:
def billions_formatter(x, pos):
    return f'{x / 1e9:.0f}B'  # Convert value to billions and format

# Create a new formatter using the billions_formatter function
formatter = FuncFormatter(billions_formatter)

# Plotting code
plt.figure(figsize=(10, 6))
yearly_trade_sum.plot(kind='bar', color='skyblue')
plt.title('Total Trade Value by Year')
plt.xlabel('Year')
plt.ylabel('Total Trade Value (Billions)')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(axis='y')  # Add grid lines only on the y-axis

# Apply the formatter to the y-axis
plt.gca().yaxis.set_major_formatter(formatter)

plt.tight_layout()
plt.show()

In [ ]:
unique_products_count = df.groupby('Year')['Section'].nunique()


# Industry Count Analysis
# Calculate count of unique industries within each section
industry_counts = df.groupby('Section')['HS2'].nunique()

# Calculate summary statistics for industry counts
industry_counts_summary = industry_counts.describe()

# Visualize distribution of industry counts across sections
# Example: Bar plot for industry counts
plt.figure(figsize=(10, 6))
industry_counts.plot(kind='bar', color='skyblue')
plt.title('Industry Count Distribution Across Sections')
plt.xlabel('Section')
plt.ylabel('Industry Count')
plt.xticks(rotation=45,fontsize=6)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# Print summary statistics for industry counts
print("Summary Statistics for Industry Counts:")
print(industry_counts_summary)

### 21 Industry in total
### Each unique industry (section) has at least 4.57 HS2(products)
### For how much the counts of unique industries or HS2 products vary from the mean value across different sections.
    - low std = low variation between HS2 (products) and Sections (unique industries)
    - High std = vice versa

# Assuming you have defined the trade summary DataFrame containing yearly trade sum, mean trade value, min trade value, and max trade value
# Also assuming df contains the trade data for Kenya

# Define the trade summary DataFrame (as per your provided code)
trade_summary = pd.DataFrame({
    'Mean Trade Value': mean_trade_value_kenya,})

# Plotting separate line graphs for Mean, Min, and Max trade values
fig, ax = plt.subplots(figsize=(10, 6))

# Plot Mean Trade Value
ax.plot(trade_summary.index, trade_summary['Mean Trade Value'], label='Mean Trade Value', marker='o', color='orange')


# Set labels and title
ax.set_xlabel('Year')
ax.set_ylabel('Trade Value')
ax.set_title('MeanTrade Values for Kenya (2012-2022)')

# Add legend
ax.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:

trade_summary = pd.DataFrame({
    'Min Trade Value': min_trade_value_kenya,})

# Plotting separate line graphs for Mean, Min, and Max trade values
fig, ax = plt.subplots(figsize=(10, 6))

# Plot Mean Trade Value
ax.plot(trade_summary.index, trade_summary['Min Trade Value'], label='Min Trade Value', marker='o', color='red')


# Set labels and title
ax.set_xlabel('Year')
ax.set_ylabel('Trade Value')
ax.set_title('Min Trade Values for Kenya (2012-2022)')

# Add legend
ax.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

plt.grid(True)
plt.tight_layout()
plt.show()

Distribution of Trade by Section: Analyze the distribution of trade value across different sections to identify which sectors contribute the most to overall trade. You can calculate summary statistics (e.g., mean, median, standard deviation) of trade values for each section and visualize the distribution using histograms or box plots.

Trend Analysis: Examine the trend in trade value over time (Year) to identify any patterns or changes in trade dynamics. Plotting the trade value against time (Year) and fitting a trend line can help visualize the overall trend and identify any significant fluctuations or trends.

Comparative Analysis: Compare trade values across different HS2 categories to identify which product categories contribute the most to overall traCan calculate summary statistics for trade values within each HS2 category and compare them to assess their relative importance.

Seasonal Analysis: If the data spans multiple years, you can perform seasonal analysis to identify any seasonal patterns or fluctuations in trade values. This can help identify peak trading seasons or periods of low activity, which may influence business decisions and strategies.

Correlation Analysis: Explore the relationship between trade value and other variables such as Section ID or Industry_Count. You can calculate correlation coefficients to quantify the strength and direction of the relationship and identify any variables that may be correlated with trade value.

Outlier Detection: Identify outliers in trade values that may indicate unusual or exceptional trading activity. You can use statistical methods such as z-scores or interquartile range (IQR) to detect outliers and investigate any potential reasons for their occurrence.

Trade Dynamics by Year: Analyze how trade dynamics have evolved over the years by comparing trade values and industry counts across different years. This can help identify trends or shifts in trade patterns over time and inform future business strategies.

By performing these analyses, you can gain a deeper understanding of the trade data and extract valuable insights to inform business decisions and strategies. Feel free to choose the analyses that are most relevant to your objectives and explore additional insights based on your specific requirements.

In [ ]:

trade_summary = pd.DataFrame({
    'Max Trade Value': max_trade_value_kenya,})

# Plotting separate line graphs for Mean, Min, and Max trade values
fig, ax = plt.subplots(figsize=(10, 6))

# Plot Mean Trade Value
ax.plot(trade_summary.index, trade_summary['Max Trade Value'], label='Max Trade Value', marker='o', color='purple')


# Set labels and title
ax.set_xlabel('Year')
ax.set_ylabel('Trade Value')
ax.set_title('Max Trade Values for Kenya (2012-2022)')

# Add legend
ax.legend()

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
correlation_df = correlation_df.drop(columns=['index'])

In [ ]:
correlation_by_section_id = df.groupby('Section ID', as_index=False).apply(lambda x: x['Trade Value'].corr(x['Year']), include_groups=False)

correlation_by_section_id

# If the correlation coefficient is close to 1, it indicates a strong positive linear relationship. This means that as one variable (e.g., 'Trade Value') increases, the other variable ('Year') also tends to increase. A correlation coefficient of 1 would indicate a perfect positive correlation.

## If the correlation coefficient is close to -1, it indicates a strong negative linear relationship. This means that as one variable increases, the other variable tends to decrease. A correlation coefficient of -1 would indicate a perfect negative correlation.

## If the correlation coefficient is close to 0, it indicates little to no linear relationship between the variables. This means that changes in one variable are not associated with systematic changes in the other variable.

Interpreting the specific correlation coefficients provided:

## Values close to 0.88, 0.37, and 0.26 suggest relatively strong positive correlations. This indicates that for these 'Section IDs', there is a strong tendency for the 'Trade Value' to increase as the 'Year' increases.

## Values close to -0.47, -0.32, and -0.27 suggest relatively strong negative correlations. This indicates that for these 'Section IDs', there is a strong tendency for the 'Trade Value' to decrease as the 'Year' increases.

## Values close to 0 or around 0.05, 0.06, etc., suggest weak correlations. This means that there is little to no linear relationship between the 'Trade Value' and 'Year' for these 'Section IDs'.

In [ ]:
# Mapping section IDs to industry names
section_id_to_industry = {
    1: 'Animal Hides',
    2: 'Animal Products',
    3: 'Animal and Vegetable Bi-Products',
    4: 'Arts and Antiques',
    5: 'Chemical Products',
    6: 'Foodstuffs',
    7: 'Footwear and Headwear',
    8: 'Instruments',
    9: 'Machines',
    10: 'Metals',
    11: 'Mineral Products',
    12: 'Miscellaneous',
    13: 'Paper Goods',
    14: 'Plastics and Rubbers',
    15: 'Precious Metals',
    16: 'Stone And Glass',
    17: 'Textiles',
    18: 'Transportation',
    19: 'Vegetable Products',
    20: 'Weapons',
    21: 'Wood Products'
}

# Map the industry names to the DataFrame
correlation_df['Industry'] = correlation_df['Section ID'].map(section_id_to_industry)

# Print the updated correlation DataFrame
correlation_df

In [ ]:
# Prepare the data for heatmap
heatmap_data = correlation_df.set_index('Industry')['Correlation Coefficient'].to_frame().T

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, annot=True, cmap='coolwarm', center=0, linewidths=.5)
sns.heatmap(heatmap_data, annot=False, ax=ax)
plt.title('Correlation Coefficients between Trade Value and Year by Industry')
plt.xlabel('Industry')
plt.ylabel('Correlation Coefficient')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
pd.read_csv("Kenya_export_destinations.csv")

In [ ]:
Export_to_country = pd.read_csv("Kenya_export_destinations.csv")

In [ ]:
Export_to_country.info()

In [ ]:
Export_to_country.mean

In [ ]:
df = pd.read_csv("Kenya_export_destinations.csv")

unique_continents = df['Continent'].unique() #dataframe of Unique_continents
num_continents = len(unique_continents) #len() gives the amount of the unique continent

print("Number of continents:", num_continents)
print("Continents:", unique_continents)

In [ ]:

trade_value_by_continent = df.groupby("Continent")["Trade Value"].sum()
print("Trade Value by Continent:\n", trade_value_by_continent)

In [ ]:
trade_by_continent_country = df.groupby(["Continent","Country"])["Trade Value"].agg([min, max])
print("Trade Value by Continent and Country\n:", trade_by_continent_country)

In [ ]:
# Find the row with maximum trade value for each continent
max_trade_by_continent = df.loc[df.groupby("Continent")["Trade Value"].idxmax()]

# Find the row with minimum trade value for each continent
min_trade_by_continent = df.loc[df.groupby("Continent")["Trade Value"].idxmin()]

# Print the results
print("Country with the Most Trade Value by Continent:", max_trade_by_continent)
print("Country with the Least Trade Value by Continent:", min_trade_by_continent)


In [ ]:
# Group the DataFrame by "Continent" and calculate the sum of trade values for each continent
trade_value_by_continent = df.groupby("Continent")["Trade Value"].sum()

# Plot the trade value distribution across continents
plt.figure(figsize=(10, 6))
trade_value_by_continent.plot(kind='line', color='skyblue')
plt.title('Trade Value Distribution Across Continents')
plt.xlabel('Continent')
plt.ylabel('Total Trade Value')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Find the row with maximum trade value for each continent
max_trade_by_continent = df.loc[df.groupby("Continent")["Trade Value"].idxmax()]

# Find the row with minimum trade value for each continent
min_trade_by_continent = df.loc[df.groupby("Continent")["Trade Value"].idxmin()]

# Extract country with the most and least trade value, along with the year
summary_max_trade = max_trade_by_continent[['Continent', 'Country', 'Trade Value', 'Year']]
summary_min_trade = min_trade_by_continent[['Continent', 'Country', 'Trade Value', 'Year']]

# Print the summary
print("Summary of Country with the Most Trade Value by Continent:")
print(summary_max_trade)

print("\nSummary of Country with the Least Trade Value by Continent:")
print(summary_min_trade)


In [ ]:
pd.read_csv("Kenya_goods_import_2012-2022_HS2.csv")

In [ ]:
Import_to_country=pd.read_csv("Kenya_goods_import_2012-2022_HS2.csv")
df = pd.read_csv("Kenya_goods_import_2012-2022_HS2.csv")


# Assuming df contains the trade data for Kenya

# Group the data by 'Year' and calculate the sum, mean, minimum, and maximum of the trade values for each year
yearly_trade_sum = df.groupby('Year')['Trade Value'].sum()
mean_trade_value_kenya = df.groupby('Year')['Trade Value'].mean()
min_trade_value_kenya = df.groupby('Year')['Trade Value'].min()
max_trade_value_kenya = df.groupby('Year')['Trade Value'].max()

# Create a DataFrame to hold the trade summary information
trade_summary = pd.DataFrame({
    'Yearly Trade Sum': yearly_trade_sum,
    'Mean Trade Value': mean_trade_value_kenya,
    'Min Trade Value': min_trade_value_kenya,
    'Max Trade Value': max_trade_value_kenya
})

# Print the trade summary DataFrame
print(trade_summary)

# Print the mean trade value for Kenya over the entire period
print("Mean Trade Value for Kenya (2012-2022):", mean_trade_value_kenya.mean())


In [ ]:
def billions_formatter(x, pos):
    return f'{x / 1e9:.0f}B'  # Convert value to billions and format

# Create a new formatter using the billions_formatter function
formatter = FuncFormatter(billions_formatter)

# Plotting code
plt.figure(figsize=(10, 6))
yearly_trade_sum.plot(kind='bar', color='skyblue')
plt.title('Total Trade Value by Year')
plt.xlabel('Year')
plt.ylabel('Total Trade Value (Billions)')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(axis='y')  # Add grid lines only on the y-axis

# Apply the formatter to the y-axis
plt.gca().yaxis.set_major_formatter(formatter)

plt.tight_layout()
plt.show()

In [ ]:
df_import = pd.read_csv("Kenya_goods_import_2012-2022_HS2.csv")


correlation_by_section_id = df.groupby('Section ID').apply(lambda x: x['Trade Value'].corr(x['Year']))




# Print the correlation coefficient for each section ID
print(correlation_by_section_id)


In [ ]:
section_id_to_industry = {
    1: 'Animal Hides',
    2: 'Animal Products',
    3: 'Animal and Vegetable Bi-Products',
    4: 'Arts and Antiques',
    5: 'Chemical Products',
    6: 'Foodstuffs',
    7: 'Footwear and Headwear',
    8: 'Instruments',
    9: 'Machines',
    10: 'Metals',
    11: 'Mineral Products',
    12: 'Miscellaneous',
    13: 'Paper Goods',
    14: 'Plastics and Rubbers',
    15: 'Precious Metals',
    16: 'Stone And Glass',
    17: 'Textiles',
    18: 'Transportation',
    19: 'Vegetable Products',
    20: 'Weapons',
    21: 'Wood Products'
}

# Plot the correlation coefficients
plt.figure(figsize=(10, 6))
plt.bar(correlation_by_section_id.index, correlation_by_section_id.values)
plt.xlabel('Industry')
plt.ylabel('Correlation Coefficient')
plt.title('Correlation Coefficients between Trade Value and Year for each Industry')
plt.xticks(ticks=correlation_by_section_id.index, labels=[section_id_to_industry.get(sec_id, 'Unknown') for sec_id in correlation_by_section_id.index], rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
pd.read_csv("GDP_per_capita.csv")

In [ ]:
# Load GDP per capita data
gdp_per_capita_df = pd.read_csv("GDP_per_capita.csv")
# Filter GDP per capita data for the years 2012-2022 using a query
# Filter GDP per capita data for the years 2012-2022 using boolean indexing
gdp_per_capita_df_filtered = gdp_per_capita_df[(gdp_per_capita_df['Year'] >= 2012) & (gdp_per_capita_df['Year'] <= 2022)]
df = gdp_per_capita_df_filtered
plt.figure(figsize=(10, 6))
plt.plot(gdp_per_capita_df_filtered['Year'], gdp_per_capita_df_filtered['GDP Per Capita'], marker='o', linestyle='-')
plt.xlabel('Year')
plt.ylabel('GDP Per Capita (USD)')
plt.title('GDP Per Capita from 2012 to 2022')
plt.grid(True)

def format_usd(x, pos):
    return f"${x:.2f}"
plt.savefig('kenya_gdp_per_cap_plot.png')
formatter = FuncFormatter(format_usd)
plt.gca().yaxis.set_major_formatter(formatter)

plt.show()
print(df)

In [ ]:
pd.read_csv("GDP.csv")

In [ ]:
# Read the GDP data from CSV
gdp_data = pd.read_csv("GDP.csv")

# Drop rows with NaN values in 'Country Name' column
gdp_data = gdp_data.dropna(subset=['Country Name'])

# Melt the dataframe to convert it into a single column for year and GDP value
gdp_data_melted = gdp_data.melt(id_vars=['Country Name', 'Country Code', 'Series Name', 'Series Code'],
                                var_name='Year',
                                value_name='GDP Value')

# Extract numeric values from the 'Year' column
gdp_data_melted['Year'] = gdp_data_melted['Year'].str.extract('(\d+)').astype(float)

# Remove rows with NaN values in 'GDP Value' column
cleaned_gdp_data = gdp_data_melted.dropna(subset=['GDP Value'])

# Filter the DataFrame to include only the desired columns and rows where the "Series Name" is "GDP (current US$)"
relevant_data = cleaned_gdp_data[(cleaned_gdp_data['Series Name'] == 'GDP (current US$)')]
relevant_data = relevant_data.drop(33)
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(range(2012, 2023), relevant_data['GDP Value'].values.flatten() / 1e9, marker='o', linestyle='-')
plt.xlabel('Year')
plt.ylabel('GDP (current US$)')
plt.title('GDP of Kenya from 2012 to 2022')
plt.grid(True)
plt.text(2012, relevant_data['GDP Value'].iloc[-1] / 1e9, 'Values in Billions', fontsize=10, verticalalignment='bottom')

plt.xticks(range(2012, 2023), rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent clipping of labels
# Save the plot as a picture file format (e.g., PNG)
plt.savefig('kenya_gdp_plot.png')

# Show the plot
plt.show()

# Print the relevant data with disclaimer
print(relevant_data)